In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import csv
import smtplib
import time
import datetime


In [2]:
# connect to the website and pull data
URL = "https://www.amazon.ca/s?k=tennis+racket&i=sporting&rh=n%3A2242989011%2Cp_89%3ABabolat%7CHEAD%7CWilson%7CYONEX&dc&crid=13LKS7TIUJ9MY&qid=1687921779&rnid=7590290011&sprefix=%2Caps%2C820&ref=sr_nr_p_89_4&ds=v1%3AfL6SUNSDq1PUrasaJbT6IsigSBlQ2pN3E30jyR%2F07P8"

# http://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)
main_soup = BeautifulSoup(page.content, "html.parser")
product_hrefs = main_soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})


In [3]:
product_links = []

for product_href in product_hrefs:
    product_links.append("https://amazon.ca" + product_href.get('href'))

#product_link = links[0].get('href')


In [4]:
def check_link(soup):
    try:
        sub_page = requests.get(soup, headers=headers)
        soup2 = BeautifulSoup(sub_page.content, "html.parser")
        return ""

    except:
        return "bad link"


In [5]:
def get_title(soup):
    try:
        return soup.find("span", attrs = {'id': 'productTitle'}).text.strip()
    except:
        return ""
        
        
def get_price(soup):
    try:    
        return soup.find("span", attrs = {'class': 'a-price aok-align-center'}).find("span", attrs = {'class': 'a-offscreen'}).text.strip()
    except:    
        return ""
        
        
def get_stock(soup):
    try:
        return soup.find("span", attrs = {'class': 'a-size-medium a-color-success'}).text.strip()
    except:
        return ""

def get_brand(soup):
    if "head" in get_title(soup).lower():
        return "Head"
    elif "wilson" in get_title(soup).lower():
        return "Wilson"
    elif "babolat" in get_title(soup).lower():
        return "Babolat"
    elif "yonex" in get_title(soup).lower():
        return "Yonex"
    elif get_title(soup) == "":
        return "Title not found"
    else:
        return "Other Brand"

def get_rating(soup):
    try:
        return (soup.find("i", attrs = {'class': 'a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big'}).text.strip())
    except:
        return ""

In [6]:
products = []

for link in product_links:
    sub_page = requests.get(link, headers=headers)
    soup2 = BeautifulSoup(sub_page.content, "html.parser")
    product = {
    "d_title": get_title(soup2),
    "d_brand": get_brand(soup2),
    "d_price": get_price(soup2),
    "d_rating": get_rating(soup2),
    "d_stock": get_stock(soup2),
    "d_link": link}
    products.append(product)
    

In [7]:
products

[{'d_title': 'HEAD Ti. Conquest Tennis Racket - Pre-Strung Head Light Balance 27 Inch Racquet - 4 3/8 in Grip',
  'd_brand': 'Head',
  'd_price': '$38.25',
  'd_rating': '4.3 out of 5 stars',
  'd_stock': 'In Stock',
  'd_link': 'https://amazon.ca/HEAD-Ti-Conquest-Tennis-Racket/dp/B095RQ6FWR/ref=sr_1_1?crid=13LKS7TIUJ9MY&keywords=tennis+racket&qid=1687978156&refinements=p_89%3ABabolat%7CHEAD%7CWilson%7CYONEX&rnid=7590290011&s=sports&sprefix=%2Caps%2C820&sr=1-1'},
 {'d_title': 'Head Tour Pro Tennis Racket - Pre-Strung Head Light Balance 27 Inch Racquet - 4 3/8 in Grip, Yellow',
  'd_brand': 'Head',
  'd_price': '$49.99',
  'd_rating': '4.5 out of 5 stars',
  'd_stock': 'In Stock',
  'd_link': 'https://amazon.ca/HEAD-Tour-Pro-Tennis-Racket/dp/B09MWPHMC4/ref=sr_1_2?crid=13LKS7TIUJ9MY&keywords=tennis+racket&qid=1687978156&refinements=p_89%3ABabolat%7CHEAD%7CWilson%7CYONEX&rnid=7590290011&s=sports&sprefix=%2Caps%2C820&sr=1-2'},
 {'d_title': 'Wilson Tour Slam Red/Black, 4 3/8"',
  'd_brand':

In [8]:
def write_dict_list_to_csv(data_list, filename):
    # Extract column names from the keys of the first dictionary
    fieldnames = list(data_list[0].keys())

    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write each dictionary as a row in the CSV file
        writer.writerows(data_list)

    print(f"Data stored in '{filename}'.")


In [11]:
write_dict_list_to_csv(products, "Output_2_Tennis_Racquets_Page_1_only.csv")

Data stored in 'Output_2_Tennis_Racquets_Page_1_only.csv'.
